In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np


In [3]:

# 1. 데이터 준비
# scikit-learn에서 Iris 데이터셋 불러오기
iris = load_iris()
X = iris.data
y = iris.target

In [5]:


# 데이터 표준화 (Normalization)
# 신경망 학습 성능 향상을 위해 각 특성의 평균을 0, 분산을 1로 만듭니다.
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [6]:

# 학습(train) 데이터와 테스트(test) 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# NumPy 배열을 PyTorch 텐서로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


In [8]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [61]:

# 2. 모델 정의 (다층 퍼셉트론, MLP)
# Iris 데이터는 4개의 특성(feature)을 가지므로 입력 레이어는 4개 노드,
# 3개 클래스로 분류하므로 출력 레이어는 3개 노드를 가집니다.
#IrisClassifier는 nn.Module을 상속받아 신경망 모델을 정의합니다

class IrisClassifier(nn.Module):
    def __init__(self):
        super(IrisClassifier, self).__init__() #부모 생성자 호출하기
        self.relu = nn.ReLU() #활성화 함수 
        self.fc1 = nn.Linear(4, 16)  # 입력(4) -> 은닉층(16)
        self.fc2 = nn.Linear(16, 32) # 은닉층(16) -> 은닉층(16)
        self.fc2_2 = nn.Linear(32, 64) # 은닉층(16) -> 은닉층(16)
        self.fc3 = nn.Linear(64, 3)  # 은닉층(16) -> 출력(3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc2_2(x)
        x = self.relu(x)
        x = self.fc3(x)  #소프트맥스함수 필요 없음
        return x

In [62]:
# 모델, 손실 함수, 옵티마이저 초기화
model = IrisClassifier()
criterion = nn.CrossEntropyLoss()
"""
nn.CrossEntropyLoss는 다음과 같은 기능을 하나로 묶어 제공합니다.

소프트맥스(Softmax): 모델의 최종 출력값(로짓, logit)을 
각 클래스에 대한 확률 분포로 변환합니다.
로그(Log): 확률값에 로그를 취합니다.
음의 로그 가능도(Negative Log Likelihood) 손실 계산: 정답 클래스에 대한 확률의 로그값에 음수를 취해 손실을 계산합니다.
"""
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [63]:
# 3. 모델 학습
def train_model(epochs):
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()  # 그라디언트 초기화
            outputs = model(inputs) # 순전파
            loss = criterion(outputs, labels) # 손실 계산
            loss.backward() # 역전파
            optimizer.step() # 가중치 업데이트
        
        # 에포크마다 손실 출력
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    print('학습 완료!')

In [64]:









# 4. 모델 평가 (학습 데이터셋 포함)
def evaluate_model():
    model.eval()  # 모델을 평가 모드로 전환
    with torch.no_grad(): # 그라디언트 계산 비활성화
        
        # 학습 데이터셋 정확도 계산
        correct_train = 0
        total_train = 0
        for inputs, labels in train_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        
        accuracy_train = 100 * correct_train / total_train
        print(f'훈련 데이터셋 정확도: {accuracy_train:.2f}%')
        
        # 테스트 데이터셋 정확도 계산
        correct_test = 0
        total_test = 0
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
        
        accuracy_test = 100 * correct_test / total_test
        print(f'테스트 데이터셋 정확도: {accuracy_test:.2f}%')

# 학습 및 평가 실행
if __name__ == '__main__':
    train_model(epochs=100)
    evaluate_model()

Epoch [1/100], Loss: 0.8178
Epoch [2/100], Loss: 0.3653
Epoch [3/100], Loss: 0.2008
Epoch [4/100], Loss: 0.1015
Epoch [5/100], Loss: 0.0097
Epoch [6/100], Loss: 0.0023
Epoch [7/100], Loss: 0.0809
Epoch [8/100], Loss: 0.0492
Epoch [9/100], Loss: 0.0150
Epoch [10/100], Loss: 0.0795
Epoch [11/100], Loss: 0.0156
Epoch [12/100], Loss: 0.0134
Epoch [13/100], Loss: 0.0001
Epoch [14/100], Loss: 0.0013
Epoch [15/100], Loss: 0.0004
Epoch [16/100], Loss: 0.0941
Epoch [17/100], Loss: 0.0255
Epoch [18/100], Loss: 0.0024
Epoch [19/100], Loss: 0.2805
Epoch [20/100], Loss: 0.0698
Epoch [21/100], Loss: 0.0075
Epoch [22/100], Loss: 0.0040
Epoch [23/100], Loss: 0.1890
Epoch [24/100], Loss: 0.5229
Epoch [25/100], Loss: 0.0047
Epoch [26/100], Loss: 0.0163
Epoch [27/100], Loss: 0.1439
Epoch [28/100], Loss: 0.2192
Epoch [29/100], Loss: 0.0030
Epoch [30/100], Loss: 0.0135
Epoch [31/100], Loss: 0.0369
Epoch [32/100], Loss: 0.0013
Epoch [33/100], Loss: 0.0001
Epoch [34/100], Loss: 0.0004
Epoch [35/100], Loss: 0